In [23]:
%cd ..

[Errno 2] No such file or directory: 'delab'
/home/pauline


In [24]:
import pandas as pd

In [50]:
from util.sql_switch import get_query_native

authors_tweets = get_query_native(
    "SELECT ta.id as author_id, conversation_id, screen_name , is_climate_author, climate_author_type FROM delab_tweet t, delab_tweetauthor ta WHERE t.author_id=ta.twitter_id")
clauthors_tweets = authors_tweets.loc[authors_tweets['is_climate_author']== True]

clauthors_tweets = clauthors_tweets.groupby(["conversation_id", "author_id"]).first()
c_cam = get_query_native(
    "SELECT author_id, c.conversation_id, depth, branching_factor, root_dominance, centrality, baseline_vision, n_posts FROM delab_conversation c, delab_conversationauthormetrics cam WHERE c.conversation_id=cam.conversation_id")

c_cam = c_cam.groupby(["author_id", "conversation_id"]).mean()
#c_cma.head()

analysis = clauthors_tweets.join(c_cam)
analysis.head(10)

using postgres
using postgres


/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,,screen_name,is_climate_author,climate_author_type,depth,branching_factor,root_dominance,centrality,baseline_vision,n_posts
conversation_id,author_id,,,,,,,,,
1575843619721707521,93,MichaelEMann,True,scientist,4.0,7.0,0.125000,5.000000,0.500000,1.0
1575916410295009308,1,mnyomb1,True,activist,3.0,5.0,0.166667,3.500000,0.500000,1.0
1576814308427644928,95,Greenpeace,True,non-governmental organisation,4.0,8.0,0.111111,5.666667,0.500000,1.0
1577040800994983936,95,Greenpeace,True,non-governmental organisation,3.0,14.0,0.066667,12.500000,0.500000,1.0
1577063450542759936,95,Greenpeace,True,non-governmental organisation,2.0,6.0,0.142857,6.000000,0.500000,1.0
1577469134119784448,93,MichaelEMann,True,scientist,2.0,6.0,0.142857,6.000000,0.500000,1.0
1577945023815065602,33,UNFCCC,True,organisation,3.0,51.0,0.019231,49.500000,0.500000,1.0
1578084206940487680,1,mnyomb1,True,activist,NaN,NaN,NaN,NaN,NaN,NaN
1578218562170155009,95,Greenpeace,True,non-governmental organisation,3.0,5.0,0.166667,3.500000,0.500000,1.0


In [ ]:
import matplotlb.pyplot as plt